In [2]:
import ee
ee.Authenticate(auth_mode='notebook')


True

In [3]:
import ee
ee.Initialize()
img = ee.Image("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")
print(img.getInfo())


EEException: Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access